In [7]:
import torch

In [13]:
ckpt= 'LPDM Checkpoints/lpdm_lol.ckpt'
checkpoint = torch.load(ckpt)

KeyboardInterrupt: 

In [14]:
for key in checkpoint['state_dict']:
    print(key)

betas
alphas_cumprod
alphas_cumprod_prev
sqrt_alphas_cumprod
sqrt_one_minus_alphas_cumprod
log_one_minus_alphas_cumprod
sqrt_recip_alphas_cumprod
sqrt_recipm1_alphas_cumprod
posterior_variance
posterior_log_variance_clipped
posterior_mean_coef1
posterior_mean_coef2
model.diffusion_model.time_embed.0.weight
model.diffusion_model.time_embed.0.bias
model.diffusion_model.time_embed.2.weight
model.diffusion_model.time_embed.2.bias
model.diffusion_model.input_blocks.0.0.weight
model.diffusion_model.input_blocks.0.0.bias
model.diffusion_model.input_blocks.1.0.in_layers.0.weight
model.diffusion_model.input_blocks.1.0.in_layers.0.bias
model.diffusion_model.input_blocks.1.0.in_layers.2.weight
model.diffusion_model.input_blocks.1.0.in_layers.2.bias
model.diffusion_model.input_blocks.1.0.emb_layers.1.weight
model.diffusion_model.input_blocks.1.0.emb_layers.1.bias
model.diffusion_model.input_blocks.1.0.out_layers.0.weight
model.diffusion_model.input_blocks.1.0.out_layers.0.bias
model.diffusion_mode

In [15]:
ckpt2 = torch.load('checkpoints/diffusion/v2-1_512-ema-pruned.ckpt')

In [16]:
for key in ckpt2['state_dict']:
    print(key)

betas
alphas_cumprod
alphas_cumprod_prev
sqrt_alphas_cumprod
sqrt_one_minus_alphas_cumprod
log_one_minus_alphas_cumprod
sqrt_recip_alphas_cumprod
sqrt_recipm1_alphas_cumprod
posterior_variance
posterior_log_variance_clipped
posterior_mean_coef1
posterior_mean_coef2
model.diffusion_model.time_embed.0.weight
model.diffusion_model.time_embed.0.bias
model.diffusion_model.time_embed.2.weight
model.diffusion_model.time_embed.2.bias
model.diffusion_model.input_blocks.0.0.weight
model.diffusion_model.input_blocks.0.0.bias
model.diffusion_model.input_blocks.1.0.in_layers.0.weight
model.diffusion_model.input_blocks.1.0.in_layers.0.bias
model.diffusion_model.input_blocks.1.0.in_layers.2.weight
model.diffusion_model.input_blocks.1.0.in_layers.2.bias
model.diffusion_model.input_blocks.1.0.emb_layers.1.weight
model.diffusion_model.input_blocks.1.0.emb_layers.1.bias
model.diffusion_model.input_blocks.1.0.out_layers.0.weight
model.diffusion_model.input_blocks.1.0.out_layers.0.bias
model.diffusion_mode